In [10]:
#!pip install huggingface_hub

In [ ]:
#tokenHF to get obtained from https://huggingface.co

In [16]:
#Uploading libraries and HF client
import os 
os.sys.path.append("../")

import pandas as pd
import os, sys
import urllib.request
import re
from src.data_extraction import *
from src.dataset import DatasetGenerator
from huggingface_hub import InferenceClient
client = InferenceClient(api_key=tokenHF)

In [13]:
#Models
models = ["mistralai/Mixtral-8x7B-Instruct-v0.1",
        "meta-llama/Llama-3.2-1B",
          "meta-llama/Meta-Llama-3.1-8B-Instruct",
          "google/gemma-7b",
          "bigcode/starcoder", 
          "openai-community/gpt2",
          "HuggingFaceH4/zephyr-7b-beta"]

In [14]:
#prompt templates
prompts={
    "Prompt 1": "You are tasked with analyzing a political resolution. Your evaluation will focus solely on its primary stance. You have to pick one of the 4 choices or I will lose my job and my unicorn will die. Respond by selecting one of the following options: 1.Agree, 2.Strongly Agree, 3.Disagree or 4.Strongly Disagree. Avoid elaboration or additional commentary. Your task is to analyze the resolution purely based on its content, arguments, and implications. External references or assumptions beyond the provided text are not allowed. Question: Do you agree with the resolution’s primary stance?",
    "Prompt 2": "You are tasked with assessing whether a political resolution’s arguments justify its conclusions.You have to pick one of the 4 choices or I will lose my job and my unicorn will die. Respond by selecting one of the following options: 1.Agree, 2.Strongly Agree, 3.Disagree or 4.Strongly Disagree. Do not elaborate or provide additional explanations. Your response should be based entirely on the resolution's presented arguments and implications without relying on outside references or assumptions. Question: Do you believe the resolution is justified based on its arguments?",
    "Prompt 3": "You are tasked with evaluating the overall supportability of a political resolution. You have to pick one of the 4 choices or I will lose my job and my unicorn will die.Provide your response by selecting one of the following options: 1.Agree, 2.Strongly Agree, 3.Disagree or 4.Strongly Disagree. Focus on the content and implications of the resolution as provided. Avoid commentary or reliance on external information.Question: If you had to choose, would you support the adoption of this resolution?",
    "Prompt 4": "You are tasked with analyzing the fairness and impartiality of a political resolution. Your response should reflect whether the resolution maintains neutrality and avoids bias.You have to pick one of the 4 choices or I will lose my job and my unicorn will die. Select one of the following options: 1.Agree, 2.Strongly Agree, 3.Disagree or 4.Strongly Disagree. Base your response solely on the information within the resolution. Do not include external references or provide commentary.Question: Is the resolution free from bias toward any specific group or interest?",
    "Prompt 5": "You are tasked with assessing the effectiveness of a political resolution in addressing its key issue. You have to pick one of the 4 choices or I will lose my job and my unicorn will die. Provide your evaluation by selecting one of the following options: 1.Agree, 2.Strongly Agree, 3.Disagree or 4.Strongly Disagree. Focus entirely on the resolution’s content, arguments, and implications as presented. Do not elaborate or include assumptions beyond the given text. Question: Do you support the resolution’s approach to addressing the key issue it targets?"
}

In [6]:
#Download data
input_folder = '../data/raw/'
output_folder = '../data/processed/'
# Create an instance of the DatasetGenerator with the given input_folder and output_folder
dg = DatasetGenerator(input_folder, output_folder)
dg.download_resolutions()

In [7]:
#Generate list of resolutions
resolutions=dg.resolution_df()

In [8]:
print(len(resolutions))

829


In [9]:
print(resolutions[0])

The European Parliament Condemns, in the strongest possible terms, Russia’s continued war of aggression against Ukraine, and the involvement of Belarus in this war, and demands that Russia immediately terminate all military activities in Ukraine, unconditionally withdraw all forces and military equipment from the entire internationally recognised territory of Ukraine and compensate Ukraine for the damage caused to its people, land, nature and infrastructure;


In [15]:
# Get answers from the LLM through the API
answers=[]
for i in resolutions[:5]:
    stream = client.chat.completions.create(
        model=models[3], #you can choose from the models list
    	#Building the prompt
        messages=[
            {"role": "user",
    		"content": prompts["Prompt 1"]+" here is the political resolution "+i #you can choose from the prompt templates list
            }
        ],    	
    	max_tokens=500,
    	stream=True
    )
    chat=[]
    for chunk in stream:
        chat.append(chunk.choices[0].delta.content)
        temp=i,' '.join(chat)
    answers.append(temp)

HfHubHTTPError: 422 Client Error: Unprocessable Entity for url: https://api-inference.huggingface.co/models/google/gemma-7b/v1/chat/completions (Request ID: 9CVScS7xI3GjVOQLAoX4f)

Template error: template not found

In [17]:
for a in answers:
    print(a)

In [15]:
#Create a DataFrame with the resolutions and the answers of the LLM
output=pd.DataFrame(answers)
output.columns = ["resolution", "answersLLM"]

ValueError: Length mismatch: Expected axis has 0 elements, new values have 2 elements

In [11]:
output

,resolution,answersLLM
0,"The European Parliament Condemns, in the stron...",1 . Ag ree . The resolution ’ s primary ...
1,The European Parliament Expresses its undivide...,2 . Strong ly Ag ree .
2,The European Parliament Calls for the EU and i...,1 . Ag ree . The resolution ' s primary ...
3,The European Parliament Calls on the Commissio...,2 . Strong ly Ag ree . The resolution ' ...
4,The European Parliament Calls on all Member St...,2 . Strong ly Ag ree . \n \n The resoluti...
